In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### I have designed these questions by going through the data-set to have an in-depth view of the data:

### Which country had the maximum number of cases ?
### Which province in the world has maximum number of cases?
### How many total deaths have been confirmed until now?
### Which country had the highest number of deaths?
### How much the confirmed cases of corona caused to fuel the increase in number of deaths?
### Which country had the best recovery rate? 


### Creating new dataFrame and reading the csv file 

In [ ]:
df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv', index_col=False)

#### A look at the dataFrame

In [ ]:
df.head()

### Cleaning the Data 

In [ ]:
df.isnull().any(axis=0)

In [ ]:
df.dropna(inplace=True)

#### Checking again

In [ ]:
df.isnull().any()

### Removing negative value from the confirmed column

In [ ]:
df.loc[df['Confirmed'] < 0]

In [ ]:
df=df[~(df['Confirmed'] <  0)]

#### Our final data-set

In [ ]:
df

In [ ]:
df.info()

## Which country had the maximum number of cases ?

In [ ]:
df['Confirmed'] = df['Confirmed'].astype('int32')
df['Country/Region'] = df['Country/Region'].astype('string')

In [ ]:
m  =pd.DataFrame(df.groupby(['Country/Region']).sum()['Confirmed'])
m

### Visualizing the number of confirmed cases 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.barplot(x=m.index, y= m.Confirmed, data=pd.melt(m), color='#042069')
sns.set(rc={"figure.figsize":(10, 4)})
sns.set_theme(style='whitegrid')
plt.xticks(rotation=90)
plt.xlabel('Countries', size=14)
plt.ylabel('No. of confirmed corona cases', size=14)
plt.show()

## Which province in the world has max number of cases?

In [ ]:
p = pd.DataFrame(df.groupby(['Province/State','Country/Region']).sum()['Confirmed'])
p.reset_index(inplace=True)
cases = p.sort_values(by='Confirmed', ascending=False).head(25)
cases 

### Plotting the cities having higher no. of cases of corona

In [ ]:
sns.barplot(x= cases['Province/State'] ,y= cases.Confirmed, data=pd.melt(cases), palette="dark:salmon_r")
plt.xticks(rotation=90)
sns.set(rc={"figure.figsize":(15, 4)})
sns.set_theme(style='ticks')
plt.show()

## How many total deaths have been confirmed until now?

In [ ]:
death = pd.DataFrame(df.groupby(['Country/Region','Province/State']).sum()['Deaths'])
death.reset_index(inplace=True) 
death

#### Total number of deaths till now

In [ ]:
df.Deaths.sum()

## Which country had the highest number of deaths?

In [ ]:
plt.bar(death['Country/Region'], death['Deaths'], width=0.8, color=sns.dark_palette('#f7347a'))
plt.xticks(rotation='vertical')
plt.xlabel('Countries', size=15)
plt.ylabel('No of Deaths', size=15)
plt.show()

## How much the confirmed cases of corona caused to fuel the increase in number of deaths?


In [ ]:
from scipy.stats import pearsonr
#df.Confirmed.corr(df.Deaths)
pearsonr(df['Confirmed'], df['Deaths'])

#### There is high correlation between the no. of deaths and the number of confirmed cases of Corona around the world!

In [ ]:
df.Confirmed.corr(df.Deaths)
mat = df[['Confirmed','Deaths', 'Recovered']].corr()
mat
#sns.heatmap(mat, annot=True, cmap='winter')
plt.subplots(figsize=(10,7))
sns.heatmap(mat, cmap='Blues', linecolor='white', linewidths=1)

## Which country had the best recovery rate? 

In [ ]:
df['Confirmed'] = df['Confirmed'].astype('int64')
df['Recovered'] = df['Recovered'].astype('int64')

In [ ]:
rec = pd.DataFrame(df[['Recovered', 'Confirmed']].value_counts(normalize=True)*100)
rec.reset_index(inplace=True)
rec.rename(columns = {0:'Rate'}, inplace=True)
rec

In [ ]:
rec['Country/Region'] = df['Country/Region'].copy()
rec

In [ ]:
rec = rec.groupby('Country/Region').sum()
rec.reset_index(inplace=True)
rec[['Recovered', 'Confirmed']].value_counts(normalize=True)*100
rec

In [ ]:
rec['Country/Region'] = rec['Country/Region'].astype('str')

### Plotting the recovery rate across all countries

In [ ]:
sns.lineplot(x= rec['Country/Region'], y= rec.Rate, data=rec)
sns.set_theme(style='darkgrid')
plt.xticks(rotation=90)
plt.show()

### Mainland China had the best recovery rate 